# Nearest Neighbors with Keras

In [ ]:
from functools import reduce
from pathlib import Path
import keras
import numpy as np
#from keras.applications import resnet50
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers import Dense, Reshape
from keras.models import Model

import numpy as np
from tqdm import tqdm_notebook as tqdm
import glob

## Base Network for Feature Extraction

In [ ]:
# Load the pretrained model which will be used to extract features.
# Note that we specify 'include_top=False'. This ensures that we don't load the final
# layers specific to the classes the model was originall trained to predict.
vgg_model = VGG16(input_shape=(224,224,3), weights='imagenet', include_top=False)
vgg_model.output.shape

## Extract features for all images in the database

In [ ]:
img_folder = Path("~/Desktop/Images/").expanduser()
img_folder_test = Path("~/Desktop/images_test/").expanduser()
image_filenames = sorted(glob.glob(str(img_folder / '*.jpg')))
len(image_filenames)

In [ ]:
def load_encode_images(encoder, filenames):
    batch_size = 16
    encoded_dim = np.prod(encoder.output.shape[1:]).value
    file_count = len(filenames)
    encoded = np.zeros((file_count, encoded_dim))
    for start_index in tqdm(list(range(0, file_count, batch_size))):
        end_index = min(start_index + batch_size, file_count)
        batch_filenames = filenames[start_index:end_index]

        batch_images = load_images(batch_filenames)
        batch_encoded = encoder.predict(batch_images)
        batch_encoded_flat = batch_encoded.reshape((len(batch_encoded), -1))
        encoded[start_index:end_index, :] = batch_encoded_flat

    return encoded

def load_images(filenames):
    images = np.zeros((len(filenames), 224, 224, 3))
    for i, filename in enumerate(filenames):
        img = image.load_img(filename, target_size=(224,224))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)
        images[i, :, :, :] = img_array
    return images

encoded_imgs = load_encode_images(vgg_model, image_filenames).T

## Build the k-NN and Joined Model

In [ ]:
def build_classifier(encoder, ref_img_coount):
    # Query vector (encoding)
    flat_dim_size = reduce(lambda x, y: x*y, vgg_model.output_shape[1:])
    query_enc_reshaped = Reshape(target_shape=(flat_dim_size,), name='query_enc_flat')(encoder.output)
    
    # Dot product between query vector and reference vectors
    x_a = Dense(units=ref_img_coount, activation='linear', name='dense_1', use_bias=False)(query_enc_reshaped)   
                
    classifier = Model(inputs=[encoder.input], outputs=x_a)
    return classifier

In [ ]:
joined_model = build_classifier(vgg_model, encoded_imgs.shape[1])
joined_model.summary()

### Normalize Encodings

In [ ]:
def normalize_ecnodings(encodings):
    ref_norms = np.sqrt(np.square(encodings).sum(axis=0)).reshape((1,encodings.shape[1]))
    return encodings / ref_norms

In [ ]:
encoded_imgs_normalized = normalize_ecnodings(encoded_imgs)

### Set Weights to Extracted Features

In [ ]:
temp_weights = joined_model.get_weights()
temp_weights[-1] = encoded_imgs_normalized
joined_model.set_weights(temp_weights)

## Predict

In [ ]:
def load_scaled_image(filename, grayscale=False, output_height=0, output_width=0, remove_border=0):
    img = kimage.load_img(filename, grayscale=grayscale)
    width, height = img.size
    img = img.crop((remove_border, remove_border, width - remove_border, height - remove_border))
    if output_height > 0 and output_width > 0:
        new_size = output_width, output_height
        img = img.resize(size=new_size)
    img_array = kimage.img_to_array(img)
    img_array = img_array.astype('float32') / 255.

    return img_array

In [ ]:
example_filename = img_folder_test / "test_burger_01.jpg"
example_img = image.load_img(example_filename, target_size=(224,224))
example_img = image.img_to_array(example_img)
example_img = np.expand_dims(example_img, axis=0)
example_img = preprocess_input(example_img)
prediction = joined_model.predict([example_img]).reshape(-1)

In [ ]:
for index in prediction.argsort()[-5:][::-1]:
    print(image_filenames[index])

## Convert to CoreML